In [3]:
# fetch usage data

endpoint = 'https://synbiohub.org/sparql?'

query = '''
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX sbh: <http://wiki.synbiohub.org/wiki/Terms/synbiohub#>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX sbol: <http://sbols.org/v2#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX purl: <http://purl.obolibrary.org/obo/>

select distinct ?pcd ?ie ?ccd
WHERE
{
?pcd a sbol:ComponentDefinition ;
sbol:component ?sc;
prov:wasDerivedFrom ?ie .

?sc sbol:definition ?ccd
}
'''

In [4]:
import requests
import urllib.parse

url = endpoint + urllib.parse.urlencode({'query': query})
print(url)
r = requests.get(url)

print(r.status_code)
print(r.encoding)
print(r.headers['content-type'])
print(r.content[0:1000])

f = open('response.xml', 'w')
f.write(r.content.decode('utf8'))

https://synbiohub.org/sparql?query=%0APREFIX+rdf%3A+%3Chttp%3A%2F%2Fwww.w3.org%2F1999%2F02%2F22-rdf-syntax-ns%23%3E%0APREFIX+dcterms%3A+%3Chttp%3A%2F%2Fpurl.org%2Fdc%2Fterms%2F%3E%0APREFIX+dc%3A+%3Chttp%3A%2F%2Fpurl.org%2Fdc%2Felements%2F1.1%2F%3E%0APREFIX+sbh%3A+%3Chttp%3A%2F%2Fwiki.synbiohub.org%2Fwiki%2FTerms%2Fsynbiohub%23%3E%0APREFIX+prov%3A+%3Chttp%3A%2F%2Fwww.w3.org%2Fns%2Fprov%23%3E%0APREFIX+sbol%3A+%3Chttp%3A%2F%2Fsbols.org%2Fv2%23%3E%0APREFIX+xsd%3A+%3Chttp%3A%2F%2Fwww.w3.org%2F2001%2FXMLSchema%23%3E%0APREFIX+rdfs%3A+%3Chttp%3A%2F%2Fwww.w3.org%2F2000%2F01%2Frdf-schema%23%3E%0APREFIX+purl%3A+%3Chttp%3A%2F%2Fpurl.obolibrary.org%2Fobo%2F%3E%0A%0Aselect+distinct+%3Fpcd+%3Fie+%3Fccd%0AWHERE%0A%7B%0A%3Fpcd+a+sbol%3AComponentDefinition+%3B%0Asbol%3Acomponent+%3Fsc%3B%0Aprov%3AwasDerivedFrom+%3Fie+.%0A%0A%3Fsc+sbol%3Adefinition+%3Fccd%0A%7D%0A
200
utf-8
application/sparql-results+xml; charset=utf-8
b'<sparql xmlns="http://www.w3.org/2005/sparql-results#" xmlns:xsi="http://www.w3.org/

14392425

In [5]:
f = open('response.xml', 'r')
content = f.read()

In [6]:
from xml.etree import ElementTree


# extract usage data
def create_usages(xml_content):
    usages = {}
    
    ns = {'sparql_results': 'http://www.w3.org/2005/sparql-results#'}
    
    root = ElementTree.fromstring(xml_content)
    results = root.find('sparql_results:results', ns)

    for result in results.findall('sparql_results:result', ns):
        bindings = result.findall('sparql_results:binding', ns)

        for binding in bindings:
            if binding.attrib['name'] == 'pcd':
                pcd = binding.find('sparql_results:uri', ns).text

        for binding in bindings:
            if binding.attrib['name'] == 'ccd':
                ccd = binding.find('sparql_results:uri', ns).text

        if pcd not in usages:
            usages[pcd] = []
        usages[pcd].append(ccd)
    
    return usages


usages = create_usages(content)

In [8]:
# usage data sanity check
print(usages['https://synbiohub.org/public/igem/BBa_K1444010/1'])

['https://synbiohub.org/public/igem/BBa_K823003/1', 'https://synbiohub.org/public/igem/BBa_R0052/1', 'https://synbiohub.org/public/igem/BBa_K090505/1']


In [2]:
# run pagerank